# 03 - Federated Learning

## Defines

Define the available types of federated learning.

In [1]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'STRATIFIED'
NUM_OF_STRATIFIED_CLIENTS = 10 # only applies to stratified method

Include the defines for the dataframe columns and the attack labels and their mappings

In [2]:
from includes import X_columns, y_column, dict_34_classes, dict_8_classes, dict_2_classes

##  Imports

In [3]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [4]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [5]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.24.2
torch 1.13.1
Training on cuda:0


## Load the Dataset

In [6]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [7]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    ######################
    # HACK TEMP CODE
    ######################
    # Set training_sets to the last entry of training_sets
    training_sets = training_sets[-1:]
    print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    ######################
    # HACK END TEMP CODE
    ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


Training sets: 135
Test sets: 34
HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - ['part-00134-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv']
Reading training data...


100%|██████████| 1/1 [00:00<00:00,  1.51it/s]

Writing training data to pickle file...
Training data size: (243649, 47)


In [8]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000838,54.62,6.05,64.00,11.961779,11.961779,0.0,0.0,0.0,0.0,...,0.111473,54.45,8.307598e+07,9.5,10.392912,0.037895,0.035900,0.02,141.55,5
1,0.005486,75.88,6.00,64.00,29.502125,29.502125,0.0,0.0,1.0,0.0,...,0.100314,54.24,8.309325e+07,9.5,10.395361,0.143036,0.346802,0.03,141.55,3
2,0.000000,0.00,45.61,65.81,151.517376,151.517376,0.0,0.0,0.0,0.0,...,57.165223,576.80,8.369379e+07,9.5,33.783684,80.958879,8638.780727,0.40,141.55,17
3,0.000000,54.00,6.00,64.00,1.500542,1.500542,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.309408e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,3
4,0.004568,745.42,5.95,65.13,8.082100,8.082100,0.0,0.0,0.0,0.0,...,549.190629,927.04,8.333561e+07,9.5,41.550978,776.661367,318084.344439,0.95,141.55,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243644,0.000000,54.00,6.00,64.00,19.582485,19.582485,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331443e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
243645,0.037146,78.22,36.21,63.18,24.542045,24.542045,0.0,0.0,0.0,0.0,...,110.233513,453.78,8.358187e+07,9.5,30.338676,154.660856,23401.960226,0.53,141.55,18
243646,3.293075,1025996.92,17.00,64.00,572.160392,572.160392,0.0,0.0,0.0,0.0,...,0.000000,554.00,8.378910e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,19
243647,0.047343,35223.00,17.00,64.00,15083.107398,15083.107398,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309852e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4


### Scale the training data input features

In [9]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

---
## Test Data
Concat the test data into a single dataframe

In [10]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    test_sets = df_sets[int(len(df_sets)*.95):]
    
    # Set training_sets to the last entry of training_sets
    test_sets = test_sets[-1:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl does not exist, constructing data...
Test sets: 1
Reading test data...


100%|██████████| 1/1 [00:00<00:00,  1.49it/s]

Writing test data to pickle file testing_data.pkl...
Testing data size: (234745, 47)


### Scale the testing data input features

In [11]:
scaler = StandardScaler()
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---


In [12]:
print("Training data size: {}".format(train_df.shape))
print("Testing data size: {}".format(test_df.shape))

Training data size: (243649, 47)
Testing data size: (234745, 47)


### Stratify the training dataset

In [13]:
# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

# take the train_df[X_columns] and split them into 10 smaller groups in the fl_X_train list using  StratifiedKFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
    fl_X_train.append(train_df[X_columns].iloc[test_index])
    fl_y_train.append(train_df[y_column].iloc[test_index])

c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


In [14]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

In [15]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

NUM_OF_ROUNDS = 10
print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()
print("Original train_df size: {}".format(train_df.shape))

print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original train_df size: (243649, 47)
Checking training data split groups
0 : X Shape (24365, 46) Y Shape (24365,)
1 : X Shape (24365, 46) Y Shape (24365,)
2 : X Shape (24365, 46) Y Shape (24365,)
3 : X Shape (24365, 46) Y Shape (24365,)
4 : X Shape (24365, 46) Y Shape (24365,)
5 : X Shape (24365, 46) Y Shape (24365,)
6 : X Shape (24365, 46) Y Shape (24365,)
7 : X Shape (24365, 46) Y Shape (24365,)
8 : X Shape (24365, 46) Y Shape (24365,)
9 : X Shape (24364, 46) Y Shape (24364,)

Checking testing data
X_test size: (234745, 46)
y_test size: (234745,)

Deploy Simulation


----


In [16]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

label = train_df[y_column]

scikit-learn 1.2.0.
flwr 1.4.0
numpy 1.24.2
tf 2.11.0


In [17]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout


class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=10, batch_size=64)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=64)
        print ("Client ", self.cid, "Evaluating complete...", accuracy, loss)
        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(len(label.unique()), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


print ("Deploy simulation...")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print ("Server Evaluating...", eval_count)
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print ("Server Evaluating complete...", accuracy, loss)
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(len(label.unique()), activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-07-08 15:18:42,827 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


scikit-learn 1.2.0.
flwr 1.4.0
numpy 1.24.2
tf 2.11.0
Deploy simulation...


2023-07-08 15:18:48,299	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-08 15:18:52,892 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 6860022988.0, 'memory': 13720045979.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 24.0}
INFO flwr 2023-07-08 15:18:52,893 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-08 15:18:52,893 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-08 15:18:57,417 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-08 15:18:57,418 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=27348) Client ID: 0
Server Evaluating... 0
7336/7336 [==============================] - 5s 738us/step


INFO flwr 2023-07-08 15:19:12,358 | server.py:91 | initial parameters (loss, other metrics): 3.512744188308716, {'accuracy': 0.013934269547462463}
INFO flwr 2023-07-08 15:19:12,359 | server.py:101 | FL starting
DEBUG flwr 2023-07-08 15:19:12,360 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


...

INFO flwr 2023-07-08 15:23:03,594 | server.py:119 | fit progress: (10, 0.20413264632225037, {'accuracy': 0.9552195072174072}, 231.23347550000108)
DEBUG flwr 2023-07-08 15:23:03,595 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 10)


Prediction:  [[1.7080903e-11 2.2275586e-10 7.0441010e-06 ... 1.0163127e-36
  1.0346972e-31 1.4476251e-31]
 [5.2131858e-11 9.6757075e-11 1.6702206e-08 ... 7.4794222e-36
  1.3233827e-29 3.0075276e-28]
 [1.3724775e-07 1.1797741e-14 3.7588378e-14 ... 2.9015667e-24
  1.0445218e-19 2.4682405e-13]
 ...
 [6.0257988e-12 1.1210903e-10 7.2082708e-05 ... 5.9792431e-36
  2.1515374e-31 1.1070483e-30]
 [1.5991832e-13 2.4862657e-11 9.4986627e-13 ... 1.2295819e-38
  6.4488143e-32 9.1913216e-30]
 [1.7147567e-13 7.3698566e-15 9.9999988e-01 ... 5.3699377e-23
  1.8645592e-21 4.5314153e-18]] (234745, 34)
Server Evaluating complete... 0.9552195072174072 0.20413264632225037
(launch_and_fit pid=30332)  [repeated 611x across cluster]
(launch_and_fit pid=15636)   1/381 [..............................] - ETA: 0s - loss: 0.1469 - accuracy: 0.9375 [repeated 39x across cluster]
(launch_and_fit pid=30332)  97/381 [======>.......................] - ETA: 0s - loss: 0.1754 - accuracy: 0.9439 [repeated 31x across cluster

DEBUG flwr 2023-07-08 15:23:04,372 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-07-08 15:23:04,373 | server.py:147 | FL finished in 232.01294979999875
INFO flwr 2023-07-08 15:23:04,374 | app.py:218 | app_fit: losses_distributed [(1, 0.4529240369796753), (2, 0.4178868234157562), (3, 0.39630854725837705), (4, 0.37226383090019227), (5, 0.34312092007480566), (6, 0.3164464056491852), (7, 0.3025070667266846), (8, 0.264240980523829), (9, 0.24068239479652787), (10, 0.17945314726750933)]
INFO flwr 2023-07-08 15:23:04,376 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-07-08 15:23:04,377 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-07-08 15:23:04,377 | app.py:221 | app_fit: losses_centralized [(0, 3.512744188308716), (1, 0.45169058442115784), (2, 0.418056458234787), (3, 0.39816784858703613), (4, 0.37313494086265564), (5, 0.35645580291748047), (6, 0.3363931179046631), (7, 0.31970104575157166), (8, 0.2995775043964386), (9, 

(launch_and_evaluate pid=35088) Client  2 Evaluating complete... 0.9543197154998779 0.1708652675151825


History (loss, distributed):
	round 1: 0.4529240369796753
	round 2: 0.4178868234157562
	round 3: 0.39630854725837705
	round 4: 0.37226383090019227
	round 5: 0.34312092007480566
	round 6: 0.3164464056491852
	round 7: 0.3025070667266846
	round 8: 0.264240980523829
	round 9: 0.24068239479652787
	round 10: 0.17945314726750933
History (loss, centralized):
	round 0: 3.512744188308716
	round 1: 0.45169058442115784
	round 2: 0.418056458234787
	round 3: 0.39816784858703613
	round 4: 0.37313494086265564
	round 5: 0.35645580291748047
	round 6: 0.3363931179046631
	round 7: 0.31970104575157166
	round 8: 0.2995775043964386
	round 9: 0.26678675413131714
	round 10: 0.20413264632225037
History (metrics, centralized):
{'accuracy': [(0, 0.013934269547462463), (1, 0.7998551726341248), (2, 0.8190845251083374), (3, 0.8267609477043152), (4, 0.854365348815918), (5, 0.8565294146537781), (6, 0.8717416524887085), (7, 0.8790432214736938), (8, 0.9015527367591858), (9, 0.9239813685417175), (10, 0.9552195072174072)]